In [136]:
# Import modules
import ee
import geemap
import geopandas as gpd

In [137]:
# Import MCDA functions
%run individual_mcda/organic_carbon.ipynb
%run individual_mcda/rainfall.ipynb
%run individual_mcda/pH.ipynb
%run individual_mcda/macro_micro_nutrients.ipynb

In [138]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [139]:
# Open source table as a dataframe
# country_gpkg = r"data\zambia_boundary.gpkg"
country_gpkg = r"C:\Users\jedid\Documents\Code\github\crop_soil_mcda\data\zambia_boundary.gpkg"
country_gdf = gpd.read_file(country_gpkg)
country_ee = geemap.gdf_to_ee(country_gdf)

# Preview the data
# country_gdf.head()

In [140]:
crop = "Maize"
rain_start = 2019
rain_end = 2025


In [ ]:
def get_mcda_image(crop, rain_start, rain_end):
    # Get individual email
    rainfall_mcda_image = get_rainfall_suitability_image(crop, rain_start, rain_end, country_ee)
    carbon_mcda_image = get_carbon_organic_suitability_image(crop, country_ee)
    ph_mcda_image = get_ph_suitability_image(crop, country_ee)
    phosphorus_mcda_image = get_nutrient_suitability_image(crop, 'phosphorus', country_ee)
    nitrogen_mcda_image = get_nutrient_suitability_image(crop, 'nitrogen', country_ee)
    potassium_mcda_image = get_nutrient_suitability_image(crop, 'potassium', country_ee)
    calcium_mcda_image = get_nutrient_suitability_image(crop, 'calcium', country_ee)
    magnesium_mcda_image = get_nutrient_suitability_image(crop, 'magnesium', country_ee)
    sulphur_mcda_image = get_nutrient_suitability_image(crop, 'sulphur', country_ee)
    zinc_mcda_image = get_nutrient_suitability_image(crop, 'zinc', country_ee)

    # Define weights for each criteria (they should sum to 1 for a normalized result)
    w_rainfall = 0.15
    w_carbon = 0.10
    w_ph = 0.10
    w_phosphorus = 0.10
    w_nitrogen = 0.15
    w_potassium = 0.10
    w_calcium = 0.10
    w_magnesium = 0.10
    w_sulphur = 0.10

    # Get MCDA Image
    mcda_image = rainfall_mcda_image \
        .add(carbon_mcda_image) \
        .add(ph_mcda_image) \
        .add(phosphorus_mcda_image) \
        .add(nitrogen_mcda_image) \
        .add(potassium_mcda_image) \
        .add(calcium_mcda_image) \
        .add(magnesium_mcda_image) \
        .add(sulphur_mcda_image)
    
    return mcda_image

In [144]:
# Visualisation parameters
mcda_vis_params = {
    'palette': ['darkred', 'red', 'yellow', 'green', 'darkgreen'],
               'min': 0, 'max': 27
        }       

In [145]:
# Visualise layers
m = geemap.Map()
m.zoom_to_gdf(country_gdf)

m.add_layer(country_ee, name= 'Zambia')

m.add_ee_layer(mcda_image, vis_params= mcda_vis_params, name= f"{crop} Suitability")
m.add_colorbar(mcda_vis_params, orientation='vertical',  
               label= f"{crop} Suitability", layer_name= f"{crop} Suitability")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…